In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt; plt.rcParams['font.size'] = 16
import sys
import pickle
import os

if 1:
    np.random.seed(1234)
    tf.set_random_seed(1234)
    RANDOM_SEED = 1234
else:
    np.random.seed(4321)
    tf.set_random_seed(4321)
    RANDOM_SEED = 4321
    
def xvfl_to_feature(arr):
    #dx
    dx = arr[:,2] - arr[:,0]
    dx = dx.reshape(-1,1)
    dx = dx[:-1,:]
    
    #dv
    dv = arr[:,3] - arr[:,1]
    dv = dv.reshape(-1,1)
    dv = dv[:-1,:]
    
    #vf and a
    vf = arr[:,1]
    a = np.diff(vf)*10
    vf = vf.reshape(-1,1)
    vf = vf[:-1,:]
    vf_later = vf[1:,:]
    a = a.reshape(-1,1)
    a.shape
    return np.hstack([dx, dv, vf,a])


with open(os.path.join('..', 'data', 'idm_data_no_restriction.pickle'),'rb') as f:
    data_pickle = pickle.load(f)

xvfl = data_pickle['idm_data'] # x, v of leading and following
para = data_pickle['para']

# random seed
np.random.seed(1234)

# test data for final evaluation
Eval_num = 10
xvfl_test = xvfl[-1*Eval_num:]

# state to feature
feature_a = list(map(xvfl_to_feature, xvfl[:-1*Eval_num]))
feature_a_in_one = np.vstack(feature_a)


# get the lb and ub
def get_lb_ub(xvfl):
    feature_a_in_one_all = np.vstack(list(map(xvfl_to_feature, xvfl)))
    lb = [feature_a_in_one_all[:,0].min(), feature_a_in_one_all[:,1].min() ,
          feature_a_in_one_all[:,2].min()]
    lb = np.array(lb)
    ub = [feature_a_in_one_all[:,0].max(), feature_a_in_one_all[:,1].max() ,
          feature_a_in_one_all[:,2].max()]
    ub = np.array(ub)
    return lb, ub
lb,ub = get_lb_ub(xvfl)
print(lb, ub)

def data_split(feature_a_in_one, DataSize, seed = RANDOM_SEED):
    feature_a_in_one = feature_a_in_one[:DataSize["total"]]# in case the total number is not consistent
    train_val_ext, test = train_test_split(feature_a_in_one, test_size = DataSize['test'], random_state = seed)
    train_val, ext = train_test_split(train_val_ext, test_size = DataSize['ext'], random_state = seed)
    train, val = train_test_split(train_val, test_size = DataSize['val'], random_state = seed)
    X_train = train[:,:3]; a_train = train[:,3].reshape(-1,1)
    X_val = val[:,:3]; a_val = val[:,3].reshape(-1,1)
    X_test = test[:,:3]; a_test = test[:,3].reshape(-1,1)
    
    # aux data
    X_ext = ext[:, :3]
    X_aux = np.concatenate([X_train, X_ext])
    return X_train, a_train, X_val, a_val, X_test, a_test, X_aux

# Training parameter
BATCH_SIZE_X1 = 256
BATCH_SIZE_X2 = 30000
EPOCH = 100

def batch_generator(X, Y, batch_size = BATCH_SIZE_X1):
    indices = np.arange(len(X)) 
    np.random.seed(seed = RANDOM_SEED)
    np.random.shuffle(indices) 
    for i in range(0, len(indices), batch_size):
        yield X[i:i+batch_size], Y[i:i+batch_size]
        
class PhysicsNN():
    def __init__(self, alpha, X_train, a_train, X_val, a_val, X_aux, para, layers, lb, ub):
        tf.set_random_seed(1234)
        self.para = para
        self.alpha = alpha
        self.X_train = X_train
        self.a_train = a_train
        self.X_val = X_val
        self.a_val = a_val
        self.X_aux = X_aux
        self.lb = lb
        self.ub = ub
        self.layers = layers
        self.history_loss = []
        self.history_loss_val = []
        self.history_partial_loss_a = []
        self.history_partial_loss_f = []
        self.history_diff_a_idm_groundtruth = []
        
        
        
        
        
        # input data
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.X_val.shape[1]])#=1, None indicates that the first dimension, corresponding to the batch size, can be of any size
        self.a_tf = tf.placeholder(tf.float32, shape=[None, self.a_val.shape[1]])
        self.x2_tf = tf.placeholder(tf.float32, shape=[None, self.X_val.shape[1]])

        # output and immediate value
        # ====== this part is for the PINN==========
        #
        #
        # ==========================================

        self.a_pred = self.net_a(self.x_tf)
        self.a_idm = self.idm_a(self.x_tf)
        self.f_pred = self.net_f(self.x2_tf) # note it should be x2

        # loss
        #self.loss_a = tf.reduce_mean(tf.square(self.a_tf - self.a_pred))
        #self.loss_f = tf.reduce_mean(tf.square(self.f_pred))

        self.loss = alpha*tf.reduce_mean(tf.square(self.a_tf - self.a_pred)) + \
                        (1-alpha)*tf.reduce_mean(tf.square(self.f_pred))
        #self.diff_a_idm_groundtruth = tf.reduce_mean(tf.square(self.a_idm - self.a_tf))
            
        
        #self.diff_a_idm_groundtruth = self.a_idm - tf.squeeze(self.a_tf)
        # optimizer
        #self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
        #                                                        method = 'L-BFGS-B', 
        #                                                        options = {'maxiter': 5000000,#Maximum number of iterations
        #                                                                   'maxfun': 5000000, #Maximum number of function evaluations
        #                                                                   'maxcor': 50, # number of limited memory matric
        #                                                                   'maxls': 50, 
        #                                                                   'ftol' : 1.0 * np.finfo(float).eps})
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        # run session of initilization
        init = tf.global_variables_initializer() # after this variables hold the values you told them to hold when you declare them will be made
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        self.sess.run(init) # initialization run
        
    def initialize_NN(self, layers):     
        tf.set_random_seed(1234)
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1): # need the -1 because the first and last number in defining the nn is input and output
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    def xavier_init(self, size):
        tf.set_random_seed(1234)
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases): # NP
        tf.set_random_seed(1234)
        
        num_layers = len(weights) + 1 # need +1, because # of weights is one dim smaller
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0  # this is all element-wise operation, centralize and standardize the input data
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        #Y = tf.exp(tf.add(tf.matmul(H, W), b)) # the final layer has no activation func, so it has to be separetly processed
        Y = tf.add(tf.matmul(H, W), b)

        return Y
    
    def net_a(self, X):
        tf.set_random_seed(1234)
        a = self.neural_net(X, self.weights, self.biases)
        return a
    
    def idm_a(self, X):
        tf.set_random_seed(1234)
        dx = X[:,0]
        dv = X[:,1]
        v = X[:,2]
        para = self.para #{'v0': 30, 'T': 1.5, 's0': 2, 'a': 0.73, 'b': 1.67}
        s0 = para['s0']; v0 = para['v0']; T = para['T']
        a = para['a']; b = para['b']
        # idm equation
        s_star = s0 + T*v - v*dv/2/tf.sqrt(a*b)
        acc = a*(1-tf.pow(v/v0, 4) - tf.pow(s_star/dx,2))
        
        return tf.reshape(acc, (-1,1))
    def net_f(self, X):
        tf.set_random_seed(1234)
        a_nn = self.net_a(X)
        a_idm = self.idm_a(X)
        f = a_nn - a_idm
        return f
    
    
    def callback(self, loss):
        #print('Loss: %e, l1: %.5f, l2: %.5f' % (loss, lambda_1, np.exp(lambda_2)))
        print('Loss: %e' % (loss))
        
    def sample_aux(self):
        tf.set_random_seed(1234)
        #idx = np.random.choice(len(self.X_aux), BATCH_SIZE_X2, replace = False)
        idx = np.array([x for x in range(len(self.X_aux))])
        return self.X_aux[idx, :]
    
    def train(self, nIter):
        tf.set_random_seed(1234)
        start_time = time.time()
        #print('start_train')
        loss = []
        for it in range(nIter):
            #train_generator = batch_generator(self.X_train, self.a_train)
            #for i, (X, a) in enumerate(train_generator):
            #i = i + 1
            tf_dict = {self.x_tf: X_train,
                       self.a_tf: self.a_train,
                      self.x2_tf: self.X_aux }
            #if (it == 0):
            #    print('==================================================================================')
           #        print("X shape:{}".format(X.shape))
             #   print("aux shape:{}".format(self.X_aux.shape))
           #        print("X train shape: {}".format(slef.X_train.shape))
           #        print("X val shape: {}".format(slef.X_val.shape))

            self.sess.run(self.train_op_Adam, tf_dict)

                
             # Print
            #if i % 10 == 0: # adam training first.. WHY?
            elapsed = time.time() - start_time
            loss_value = self.sess.run(self.loss, tf_dict)
            #partial_loss_a = self.sess.run(self.loss_a, tf_dict)
            #partial_loss_f = self.sess.run(self.loss_f, tf_dict)
            #diff_a_idm_groundtruth = self.sess.run(self.diff_a_idm_groundtruth, tf_dict)
            
            
            # val loss
            #a_pred_onVal = self.predict(self.X_val)
            #loss_val = sum((a_pred_onVal-self.a_val)**2)/len(a_pred_onVal)

            #print('It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f, Time: %.2f' % (it, loss_value, 1.0, 1.0, elapsed))
            
            #print('Epoch: %d,  Loss: %.3e, Loss_val: %.3e, Time: %.2f' % 
            #      (it,  loss_value, loss_val,elapsed))
            
            
            #start_time = time.time()
            
            # cross validation
            #if it % Val_freq == 0:
            self.history_loss.append(loss_value)
            loss.append(loss_value)
            #self.history_loss_val.append(loss_val)
            #self.history_partial_loss_a.append(partial_loss_a)
            #self.history_partial_loss_f.append(partial_loss_f)
            #self.history_diff_a_idm_groundtruth.append(diff_a_idm_groundtruth)
        #print('Epoch: %d,  Loss: %.3e, Loss_val: %.3e, Time: %.2f' % 
        #          (it,  loss_value, loss_val,elapsed))
        return loss
           
                
            
            
        
        
            
            
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star}
        
        a_pred = self.sess.run(self.a_pred, tf_dict)
        #f_star = self.sess.run(self.f_pred, tf_dict)
        
        return a_pred#, f_star
    
    
        
        
        
    def predict_idm(self, X_star):
        tf_dict = {self.x_tf: X_star}
        
        a_idm = self.sess.run(self.a_idm, tf_dict)
        #f_star = self.sess.run(self.f_pred, tf_dict)
        
        return a_idm#, f_star


def simulate(init, XV_L, nn):
    xl_0 = init[2]  
    vl_0 = init[3]
    xf_0 = init[0]
    vf_0 = init[1]
    assert xl_0 == XV_L[0,0]
    assert vl_0 == XV_L[0,1]
    XF = [xf_0]
    VF = [vf_0]

    for i in range(0, len(XV_L)-1):
        one_state = np.array([XV_L[i,0], XV_L[i,1], XF[-1], VF[-1]])
        feature = np.array([one_state[0]-one_state[2], one_state[1]-one_state[3], one_state[-1]])
        a = nn.predict(feature.reshape(-1,3)) # xv_lf
        v_next = VF[-1] + 0.1 * a
        x_next = (v_next + VF[-1])/2*0.1 + XF[-1]
        XF.append(x_next.flatten()[0])
        VF.append(v_next.flatten()[0])
    return XF, VF

# relative error function
def get_XV_error(xvfl_test, nn):
    XFs = []
    VFs = []
    XF_tests = []
    VF_tests = []
    for test_idx in range(len(xvfl_test)):
        XF, VF = simulate(xvfl_test[test_idx][0,:], xvfl_test[test_idx][:,2:], nn)
        XFs.append(np.array(XF))
        VFs.append(np.array(VF))
        XF_tests.append(xvfl_test[test_idx][:,0])
        VF_tests.append(xvfl_test[test_idx][:,1])

    XFs = np.concatenate(XFs)
    VFs = np.concatenate(VFs)
    XF_tests = np.concatenate(XF_tests)
    VF_tests = np.concatenate(VF_tests)

    X_error = np.sqrt( sum(np.square(XFs-XF_tests))/sum(np.square(XF_tests)) )
    V_error = np.sqrt( sum(np.square(VFs-VF_tests))/sum(np.square(VF_tests)) )
    
    return X_error, V_error

def data_revise_nn(num_train, seed = 1234):
    np.random.seed(seed)
    DataSize = {"train": num_train,
               "ext": 30000,
               "val": int(0.4*num_train),
               "test": 30000}
    DataSize["total"] = sum(DataSize.values())
    
    idx = np.random.choice(len(feature_a_in_one), DataSize["total"] , replace = False) # feature_a_in_one is global
    feature_a_in_one_dowsample = feature_a_in_one[idx]
    X_train, a_train, X_val, a_val, X_test, a_test, X_aux = data_split(feature_a_in_one_dowsample, DataSize, seed = seed)
    
    # model
    
    #nn.X_train = X_train
    #nn.a_train = a_train
    #nn.X_val = X_val
    #nn.a_val = a_val
    #nn.X_aux = X_aux
    BATCH_SIZE_X2 = len(X_aux)  # global variable
    return X_train, a_train, X_val, a_val, X_test, a_test, X_aux
    
    


def nn_test_alpha_beta(nn):
    print('X_train shape: {}'.format(nn.X_train.shape))
    print('X_val shape: {}'.format(nn.X_val.shape))
    
    #tf.reset_default_graph()
    #nn.loss = alpha * tf.reduce_mean(tf.square(nn.a_tf - nn.a_pred)) + \
    #                  beta * tf.reduce_mean(tf.square(nn.f_pred))
    
    ## train
    #nn.train_op_Adam = nn.optimizer_Adam.minimize(nn.loss)
    #init = tf.global_variables_initializer()
    #nn.sess.run(init)
    nn.train(EPOCH)
    
    # test_mse
    a_test_star = nn.predict(X_test)
    #a_test_star.shape
    a_test_star = nn.predict(X_test)
    test_mse = np.mean(np.square(a_test_star - a_test))
    
    # X error V error
    x_error, v_error = get_XV_error(xvfl_test, nn)
    
    return test_mse, x_error, v_error

    
    
    


In [ ]:
# ga
# GA
from GA.GA import *

args_GA = {
    'sol_per_pop': 10,
    'num_parents_mating': 5,
    'num_mutations': 1, # set 1 to mutate all the parameters
    'mutations_extend': 0.1,
    'num_generations': 10,
    
    'delta_t': 0.1,
    'mse': 'position',
    'RMSPE_alpha_X': 0.5,
    'RMSPE_alpha_V': 0.5,
    'lb' : [10, 0, 0, 0, 0],
    'ub' : [40, 10, 10, 5, 5]
}



#del_module("GA")
ga = GA(args_GA)


In [ ]:
layers = [3, 10, 10, 5, 1]
Num_trains = [1000]
for i in np.arange(5000, 55000, 5000 ):
    Num_trains.append(i)
Alphas = np.arange(0,1.1,0.1)
record = dict()

for num_train in Num_trains:
    for alpha in Alphas:
        Test_mse = []
        X_error = []
        V_error = []
        GA_para = []
        for it in range(10):
            print("num_train: %d, alpha: %.1f  iter: %d"%(num_train, alpha, it))

            tf.reset_default_graph()
            start_time = time.time()
            X_train, a_train, X_val, a_val, X_test, a_test, X_aux = data_revise_nn(num_train, seed = it)
            
            #----- from X_train, a_train, get a IDM para
            para, mse, duration = ga.executeGA( X_train )
            GA_para.append(para)
            print('GA duration -> : {}'.format(time.time() - start_time))
            
            #
            nn = PhysicsNN( alpha, X_train, a_train, X_val, a_val, X_aux, para, layers, lb, ub)
            test_mse, x_error, v_error = nn_test_alpha_beta(nn)
            print('NN duration -> : {}'.format(time.time() - start_time))
            Test_mse.append(test_mse)
            X_error.append(x_error)
            V_error.append(v_error)
            break

        String = str(num_train) + ',' + str(alpha)
        record[String] = dict()
        record[String]['Test_mse'] = Test_mse
        record[String]['X_error'] = X_error
        record[String]['V_error'] = V_error
        record[String]['GA_para'] = GA_para
        #with open('alpha_curve_avg10_0426_01.pickle', 'wb') as f:
        #    pickle.dump(record, f)
        break
    break